In [ ]:
cc = Contributor.objects.annotate(bylines=Count('byline')).order_by('-bylines')
names = cc.values_list('display_name', 'bylines')

threshold = 85
other_names = [name for name, bylines in names if bylines < 5 and len(name) > 8 ]
for index, (name, bylines) in enumerate(names):
    if bylines < 5: 
        break
    #print (index, name, bylines)
    duplicates = []
    for other in other_names[:]:
        if fuzz.partial_ratio(name, other) >= threshold:
            other_names.remove(other)
            if other != name:
                duplicates.append(other)
                # cnt = cc.filter(display_name=other)[0]
                #print(cnt.legacy_data())
            
    if duplicates:
        print(name, bylines, ':', '\n      '.join(['']+duplicates))
        print()

        
        

In [ ]:
cc = Contributor.objects.annotate(bylines=Count('byline')).order_by('-bylines')
names = cc.values_list('display_name', 'bylines')
for c in cc: print( c.bylines, c.display_name, c.aliases.replace('\n', '; '),)

In [ ]:
from IPython.core import display
import re
import json
from fuzzywuzzy import fuzz
from django.db.models import Count

In [ ]:
initials = {
 'AAAH': 'Anders Aa Hagen',
 'AAH': 'Åse Holte',
 'ABE': 'Åshild Bekke Eidem',
 'AH': 'Audun Halaas',
 'AHA': 'Aurora Hannisdal',
 'AJ': 'Aslak Johannesen',
 'AO': 'Anne Ogundipe',
 'AOE': 'Amund K. Østbye',
 'AS': 'Åshild Støylen',
 'BO': 'Brian Olguin',
 'CE': 'Christine Erlbeck',
 'CK': 'Christine Karijord',
 'CT': 'Camilla Tully',
 'DC': 'Damir Cvetojevic',
 'ED': 'Ellinor Bent Dalbye',
 'EJ': 'Elisabeth Johansen',
 'FA': 'Fredrik Arff',
 'GKJ': 'Grethe Kielland-Jenssen',
 'GON': 'Gunhild Oland Nedrelid',
 'HFB': 'Harald Fagerheim Bugge',
 'HH': 'Hanne O. Hvattum',
 'HL': 'Håken Lid',
 'HM': 'Håkon Mosvold',
 'HR': 'Hilde Rustan',
 'IK': 'Ivàn Kverme',
 'IS': 'Ingjerd Sørlie',
 'JH': 'Julie Hoen',
 'KG': 'Kristine Gramstad',
 'KK': 'Kamran Karimi',
 'KMD': 'Kaj M. Duesund',
 'KRN': 'Kristian Ridder-Nielsen',
 'LF': 'Lise Falck',
 'LRL': 'Liv Ronnaug Lilleåse',
 'MF': 'Mia Frogner',
 'MH': 'Mari Hagen',
 'MLH': 'Mari Lund Haanshus',
 'MNP': 'Marius Nergård Pettersen',
 'NL': 'Niklas Lello',
 'PAW': 'Paul Anthony Wilson',
 'RE': 'Richard Eriksen',
 'RRJ': 'Robin Røkke Johansen',
 'SE': 'Silje Eide',
 'SL': 'Stéphane Lelarge',
 'SR': 'Silje Ramstad',
 'SS': 'Susanne Stensland',
 'STL': 'Sindre T. Lønnes',
 'TCST': 'Tone Celine Sundseth Thorgrimsen',
}

In [ ]:
def legacy_bylines(story):
    """ Finds original byline for story."""
    web_source = story.legacy_html_source
    prodsys_source = story.legacy_prodsys_source
    bylines = []
    if web_source:
        bylines = json.loads(web_source)[0]['fields']['byline']
    elif prodsys_source:
        content = json.loads(prodsys_source)[0]['fields']['tekst']
        bylines = '\n'.join(line for line in content.splitlines() if line.startswith('@bl:'))
    return bylines
       

def serialize_bylines(story):
    bylines = []
    for byline in story.byline_set.order_by('ordering', 'pk'):
        bylines.append(
            '{credit}: {name}{description}'.format(
                credit=byline.credit,
                name=byline.contributor.display_name,
                description='' if not byline.title else ', %s' % byline.title,
                ).replace('by: ', '')
            )
    return bylines
                
def fixed_bylines(story):
    legacy_bl = legacy_bylines(story)
    if '•' in legacy_bl:
       return legacy_bl.split('•') 
    
    if legacy_bl.upper()==legacy_bl: 
        # intialer
        bylines = legacy_bl.split('/')        
        fixed_bl = []
        for bl in bylines:
            try:
                fixed_bl.append(initials[bl])
            except KeyError:
                candidates = Contributor.objects.filter(initials=bl.strip())
                names = []
                for candidate in candidates:
                    if candidate.active_stints(story.publication_date):
                        names.append(candidate.display_name)
                if len(names) == 1:
                    fixed_bl.append(names[0])
                else:
                    return []
                    fixed_bl.append('%s (%d)' % (bl, len(names)))
                    
        return fixed_bl
    
    if story.bylines.count() > 1:
        return serialize_bylines(story)
                
    return legacy_bylines(story).splitlines() 
                
            
    
bl = 'Solveig N. Langvad Helle Gannestad'
cnt = Contributor.objects.get(display_name=bl)

print(serialize_bylines(cnt))

In [ ]:
broken_stories = Story.objects.filter(publication_status=Story.STATUS_ERROR)
broken_dict = {}
from collections import OrderedDict
for story in broken_stories:
    #print(story)
    #print(serialize_bylines(story))
    #print(legacy_bylines(story))
    
    broken_dict[story.pk] =  OrderedDict([
        ('title', story.title,),
        ('broken_bl', serialize_bylines(story),),
        ('old_bl', legacy_bylines(story)),
        ('fixed_bl', fixed_bylines(story),),
    ]) 

print(broken_dict)

In [ ]:
#with open('broken_stories.json', 'w') as fp:
#    json.dump(broken_dict, fp, indent=2, ensure_ascii=False)

In [ ]:
with open('broken_stories.json', 'r') as fp:
    broken = json.load(fp)

In [ ]:
broken = broken_dict
for key in sorted(broken.keys(), key=int):
    story = Story.objects.get(pk=int(key))
    fixed_bl = broken[key]['fixed_bl']
    if fixed_bl:
        story.byline_set.all().delete()
        new_bylines = '\n'.join('@bl:%s'%byline for byline in fixed_bl if byline)
        story.bodytext_markup += '\n' + new_bylines
        story.publication_status=Story.STATUS_PUBLISHED
        story.full_clean()
        story.save()
        print(story.pk, fixed_bl, serialize_bylines(story), sep = '\n  ')

In [ ]:
with open('broken_stories') as fp:
    broken_pks = [int(pk) for pk in fp.readlines()]

In [ ]:
repaired_bylines = {}
for pk in broken_pks:
    story = Story.objects.get(pk=pk)
    if story.publication_status != Story.STATUS_ERROR:
        print(serialize_bylines(story))
        repaired_bylines[pk] = serialize_bylines(story)

In [ ]:
with open('../repaired_bylines.json', 'w') as fp:
    json.dump(repaired_bylines, fp, indent=2, ensure_ascii=False)

In [ ]:
Story.objects.published().filter(modified__gte='2015-12-01').dump_bylines()

In [ ]:
Story.objects.load_bylines('testdump.json')

In [1]:
from apps.stories.models import broken_bylines_in_story

broken_stories = Story.objects.filter(publication_status=Story.STATUS_ERROR)

In [2]:
for story in broken_stories:
    broken_bylines_in_story(story)
    story.save()

In [ ]:
story.bodytext_markup

In [ ]:
def 